In [1]:
from datahelpers.community_tools import prepare_graphdf, prepare_graph_from_df
from datahelpers.community_tools import meta_calculate_comms

DeprecationWarning: To avoid name collision with the igraph project, this visualization library has been renamed to 'jgraph'. Please upgrade when convenient.

install datahelpes from here : https://github.com/alexander-belikov/datahelpers
also pandas and igraph should be installed

In [8]:
from itertools import product
from pprint import pprint
from pandas import HDFStore, read_hdf
from os.path import expanduser

In [9]:
versions = [8, 11]
df_types = ['lit', 'gw']

fnames = ['edges_all_{0}{1}.h5'.format(t, v) for t, v in zip(df_types, versions)]
print(fnames)

['edges_all_lit8.h5', 'edges_all_gw11.h5']


## Check the keys in hdf files

In [11]:
keys = {}
for f in fnames:
    # store = HDFStore(expanduser(f), mode='r')
    store = HDFStore(f, mode='r')
    store = read_hdf(f, '/y2013')
    print(store)
    keys[f] = sorted(store.keys())

               up     dn  0
0               2    109  1
1               2    213  3
2               2    324  2
3               2    348  4
4               2    354  3
5               2    434  1
6               2    462  4
7               2    718  1
8               2    973  3
9               2   1051  1
10              2   1147  1
11              2   1154  1
12              2   1401  2
13              2   1435  1
14              2   1509  1
15              2   1510  1
16              2   1634  1
17              2   1991  2
18              2   2048  1
19              2   2098  1
20              2   2157  1
21              2   2244  1
22              2   2247  1
23              2   2670  1
24              2   2852  1
25              2   2986  1
26              2   3309  1
27              2   3339  1
28              2   3553  1
29              2   3569  1
...           ...    ... ..
144142  100507436   3375  1
144143  100507436   3458  3
144144  100507436   3821  7
144145  100507436   

TypeError: '<' not supported between instances of 'int' and 'str'

In [5]:
print(keys)

{'edges_all_lit8.h5': ['/y1947', '/y1950', '/y1951', '/y1952', '/y1960', '/y1963', '/y1964', '/y1965', '/y1966', '/y1967', '/y1968', '/y1969', '/y1970', '/y1971', '/y1972', '/y1973', '/y1974', '/y1975', '/y1976', '/y1977', '/y1978', '/y1979', '/y1980', '/y1981', '/y1982', '/y1983', '/y1984', '/y1985', '/y1986', '/y1987', '/y1988', '/y1989', '/y1990', '/y1991', '/y1992', '/y1993', '/y1994', '/y1995', '/y1996', '/y1997', '/y1998', '/y1999', '/y2000', '/y2001', '/y2002', '/y2003', '/y2004', '/y2005', '/y2006', '/y2007', '/y2008', '/y2009', '/y2010', '/y2011', '/y2012', '/y2013'], 'edges_all_gw11.h5': ['/y1975', '/y1976', '/y1977', '/y1978', '/y1979', '/y1980', '/y1981', '/y1982', '/y1983', '/y1984', '/y1985', '/y1986', '/y1987', '/y1988', '/y1989', '/y1990', '/y1991', '/y1992', '/y1993', '/y1994', '/y1995', '/y1996', '/y1997', '/y1998', '/y1999', '/y2000', '/y2001', '/y2002', '/y2003', '/y2004', '/y2005', '/y2006', '/y2007', '/y2008', '/y2009', '/y2010', '/y2011', '/y2012', '/y2013']}


## prepare_graphdf fetches the edges from a list hdf files that are under the key, e.g. 1995

In [6]:
df = prepare_graphdf(fnames, 'edges', 1995)

NameError: name 'prepare_graphdf' is not defined

In [6]:
df.shape

(28644, 3)

In [7]:
df.head()

,up,dn,0
0,2,213,2
1,2,348,2
2,2,462,4
3,2,718,1
4,2,973,3


## prepare_graph_from_df creates a graph (that has integer nodes) and provides weights and also a map i -> actual node

In [8]:
g, weights, inv_conv = prepare_graph_from_df(df, 'edges', directed=False, percentile_value=None)